In [1]:
##Estimación de indicadores de gobierno
#Se borra todo lo que se encuentra en el entorno

#rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr)

Loading required package: pacman



In [2]:
url_encig_basica<-"https://www.inegi.org.mx/contenidos/programas/encig/"
segunda_parte<-"microdatos/encig"
years_encig<-c("2017","2019","2021", "2023")

In [3]:
# Descargar los archivos, descomprimirlos y renombrarlos
for (i in 1:length(years_encig)) {
  # Ajustar el sufijo de la URL y archivo según el año
  if (years_encig[i] %in% c("2017", "2019")) {
    fin <- "_base_datos_dbf.zip"
  } else {
    fin <- "_base_datos_csv.zip"
  }

  # Construir la URL y el nombre del archivo destino
  url <- paste0(url_encig_basica, years_encig[i], "/", segunda_parte, str_sub(years_encig[i], 3, 4), fin)
  print(url)
  destfile <- paste0("data/encig", str_sub(years_encig[i], 3, 4), fin)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = "data")
}

[1] "https://www.inegi.org.mx/contenidos/programas/encig/2017/microdatos/encig17_base_datos_dbf.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/encig/2019/microdatos/encig19_base_datos_dbf.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/encig/2021/microdatos/encig21_base_datos_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/encig/2023/microdatos/encig23_base_datos_csv.zip"


In [49]:
# Función de estimación
process_files <- function(years, variable) {
  # Función anidada para estimar el indicador
  estima_indicador <- function(path, variable) {
    # Extraer el año del archivo
    year <- substr(path, 11, 14)

    # Leer los datos y hacer las transformaciones necesarias de las variables
    if (year %in% c("2017", "2019")) {
      datos <- foreign::read.dbf(path) %>%
        janitor::clean_names() %>%
        mutate(p11_1_6 = if ("p11_1_6" %in% names(.)) as.numeric(p11_1_6) else as.numeric(p3_2),
               fac_p18 = as.numeric(fac_p18),
               upm = as.numeric(upm),
               est_dis = as.numeric(est_dis)) %>%
        # Filtrar Ciudad de México
        filter(ent == "09")
    } else if (year %in% c("2021", "2023")) {
      datos <- read_csv(path, show_col_types = FALSE) %>%
        janitor::clean_names() %>%
        rename_with(~ ifelse(. == "p11_1_06", "p11_1_6", .)) %>%
        # Filtrar Ciudad de México
        filter(nom_ent == "CIUDAD DE MEXICO")
    } else {
      stop("Año no soportado.")
    }

    # Diseño muestral
    dm <- datos %>%
      as_survey_design(ids = upm, strata = est_dis, weights = fac_p18)

    # Confianza
    if (variable == "confianza") {
      confianza_gob <- dm %>%
        mutate(confianza = case_when(
          p11_1_6 == 1 | p11_1_6 == 2 ~ "Confía",
          TRUE ~ "No confía"
        )) %>%
        group_by(confianza) %>%
        summarise(
          personas = survey_total(vartype = "cv"),
          porcentaje = survey_mean(vartype = "cv")
        ) %>%
        # Filtrar confianza
        filter(confianza == "Confía") %>%
        mutate(porcentaje = round(porcentaje * 100, 2),
               year = year)
      return(confianza_gob)

      # Actos de corrupción
    } else if (variable == "actos_corrupcion") {
      actos_corrup <- dm %>%
        mutate(actos_corrupcion = case_when(
          p3_2 == 1 | p3_2 == 2 ~ "Frecuentes",
          TRUE ~ "No frecuentes"
        )) %>%
        group_by(actos_corrupcion) %>%
        summarise(
          personas = survey_total(vartype = "cv"),
          porcentaje = survey_mean(vartype = "cv")
        ) %>%
        filter(actos_corrupcion == "Frecuentes") %>%
        mutate(porcentaje = round(porcentaje * 100, 2),
               year = year)
      return(actos_corrup)

    } else {
      stop("Variable no reconocida.")
    }
  }

  map_dfr(years, function(year) {
    # Seleccionar el archivo correcto según la variable y el año
    file_path <- if (variable == "actos_corrupcion") {
      if (year %in% c(2017, 2019)) {
        paste0("data/encig", year, "_01_sec1_3_4_5_8_9_10.dbf")
      } else if (year %in% c(2021, 2023)) {
        paste0("data/encig", year, "_01_sec1_A_3_4_5_8_9_10.csv")
      } else {
        stop("Año no soportado para actos de corrupción.")
      }
    } else if (variable == "confianza") {
      if (year %in% c(2017, 2019)) {
        paste0("data/encig", year, "_01_sec_11.dbf")
      } else if (year %in% c(2021, 2023)) {
        paste0("data/encig", year, "_01_sec_11.csv")
      } else {
        stop("Año no soportado para confianza.")
      }
    } else {
      stop("Variable no reconocida.")
    }

    # Print del archivo que se está procesando
    print(paste("Procesando archivo:", file_path))

    # Invocar la función de estimación
    estima_indicador(file_path, variable)
  })
}


In [50]:
# Estimar indicadores de gobierno
tabla_confianza<-process_files(years_encig, variable = "confianza")
print("Porcentaje de personas que confían en el gobierno de la Ciudad de México")
tabla_confianza

[1] "Procesando archivo: data/encig2017_01_sec_11.dbf"
[1] "Procesando archivo: data/encig2019_01_sec_11.dbf"
[1] "Procesando archivo: data/encig2021_01_sec_11.csv"


New names:
• `` -> `...1`


[1] "Procesando archivo: data/encig2023_01_sec_11.csv"


New names:
• `` -> `...1`


[1] "Porcentaje de personas que confían en el gobierno de la Ciudad de México"


confianza,personas,personas_cv,porcentaje,porcentaje_cv,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Confía,1241859,0.05099752,14.16,0.04852940,2017
Confía,2836364,0.02448298,40.87,0.02166087,2019
Confía,3351823,0.02201146,48.17,0.01892866,2021
Confía,3446410,0.02412965,45.49,0.01937280,2023


In [51]:
tabla_actos_corrupcion<-process_files(years_encig, variable = "actos_corrupcion")
print("Porcentaje de personas que perciben actos de corrupción en la Ciudad de México")
tabla_actos_corrupcion

[1] "Procesando archivo: data/encig2017_01_sec1_3_4_5_8_9_10.dbf"
[1] "Procesando archivo: data/encig2019_01_sec1_3_4_5_8_9_10.dbf"
[1] "Procesando archivo: data/encig2021_01_sec1_A_3_4_5_8_9_10.csv"


New names:
• `` -> `...1`


[1] "Procesando archivo: data/encig2023_01_sec1_A_3_4_5_8_9_10.csv"


New names:
• `` -> `...1`


[1] "Porcentaje de personas que perciben actos de corrupción en la Ciudad de México"


actos_corrupcion,personas,personas_cv,porcentaje,porcentaje_cv,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Frecuentes,8410975,0.01634467,95.94,0.003774120,2017
Frecuentes,6247296,0.01193104,90.02,0.005535388,2019
Frecuentes,6248752,0.01368776,89.81,0.005471011,2021
Frecuentes,6648218,0.01374265,87.76,0.006548517,2023
